# IPAD 

In [1]:
# load packages
using Revise
using Knockoffs
using LinearAlgebra
using Random
using StatsBase
using Statistics
using ToeplitzMatrices
using CSV, DataFrames
using BlockDiagonals
using Distributions
using RCall

┌ Info: Precompiling Knockoffs [878bf26d-0c49-448a-9df5-b057c815d613]
└ @ Base loading.jl:1342


## Check computation of r matches with Zihuai

In [9]:
n = 500
p = 1000
kmax = min(n, p)
μ = zeros(p)
Σ = simulate_AR1(p, a=3, b=1)
X = rand(MvNormal(μ, Σ), n)' |> Matrix
Sigma = X * X';
@rput Sigma

# Zihuai's code for computing r (operates on Sigma)
R"""
eigen.fit<-eigen(Sigma)
m<-ncol(Sigma)

#Option 1: ER
temp<-eigen.fit$values[-length(eigen.fit$values)]/eigen.fit$values[-1]
r_er <-which.max(temp[1:(m-1)])

#Option 2: GR
Get.V<-function(k){sum(eigen.fit$values[(k+1):m])}
Get.GR<-function(k){log(1+eigen.fit$values[k]/Get.V(k))/log(1+eigen.fit$values[k+1]/Get.V(k+1))}
temp<-sapply(1:(m-2),Get.GR)
r_gr <-which.max(temp)

#Option 3: variation explained
r_ve <-which(cumsum(eigen.fit$values)/sum(eigen.fit$values)>0.9)[1]
"""

# Zihuai's result
@rget r_er r_gr r_ve
r_er, r_gr, r_ve

(5, 5, 210)

In [10]:
# ER
XXt = X * X'
evals, evecs = eigen(XXt)
evecs = evecs[:, sortperm(evals)]
reverse!(evals)
r_er = [evals[i] / evals[i + 1] for i in 1:kmax-1] |> argmax

5

In [11]:
# GR
v(evals, k::Int) = sum(@views(evals[k+1:end]))
gr(evals, k::Int) = log(1 + evals[k] / v(evals, k)) / log(1 + evals[k+1] / v(evals, k+1))
r_gr = [gr(evals, k) for k in 1:kmax-1] |> argmax

5

In [12]:
# VE
r_ve = findfirst(x -> x > 0.9, cumsum(evals) ./ sum(evals))

210

### Generate knockoffs

In [6]:
ko = ipad(X, m=5)
X̃ = ko.X̃

500×5000 Matrix{Float64}:
 -0.313405   -0.0728747   1.77162     …  -1.32162    -0.551875     0.259281
 -0.37997    -0.695135   -1.33555         0.534953    0.423396     0.0975494
  0.652966    0.989096   -0.442769       -0.263564   -0.0365178    0.469957
  0.116166   -0.68938    -1.726           0.741814    0.00563501   0.577217
  0.247287   -0.191932   -0.110717        1.93746     1.00172      0.619334
  0.495798    0.559977   -1.08761     …   1.24883     1.50916      0.235961
  0.119767   -1.35437     0.181569        0.813892    0.504228     1.10472
  0.810737   -0.35596     0.146964        2.45423     1.37003      1.74249
 -0.335212   -1.15532    -0.453091       -0.690757    0.0012343    0.0360875
 -2.02369    -2.11495     1.01508        -0.0591327   1.14913      1.29991
 -0.444251   -1.76954    -1.18168     …  -1.65436    -0.344728     0.024917
  0.486889    1.93956    -0.122134       -0.552824    0.0613275   -1.08632
  0.588333   -0.0938921  -0.180687        1.03942     1.07361   

Check empirical vs true covariance

In [7]:
Σ # truth

1000×1000 Matrix{Float64}:
 1.0           0.734181      0.145804      …  1.54311e-143  1.2164e-143
 0.734181      1.0           0.198396         2.09971e-143  1.65516e-143
 0.145804      0.198396      1.0              1.05729e-142  8.33438e-143
 0.0896023     0.121922      0.613925         1.72045e-142  1.3562e-142
 0.088693      0.120685      0.607694         1.73809e-142  1.37011e-142
 0.0595926     0.0810878     0.408308      …  2.58684e-142  2.03916e-142
 0.0455027     0.0619156     0.311769         3.38786e-142  2.67058e-142
 0.039132      0.0532469     0.268119         3.93941e-142  3.10536e-142
 0.0317363     0.0431836     0.217446         4.85743e-142  3.82902e-142
 0.0239226     0.0325515     0.16391          6.44397e-142  5.07966e-142
 0.0139045     0.0189199     0.0952691     …  1.10868e-141  8.73952e-142
 0.0102667     0.0139699     0.070344         1.50152e-141  1.18362e-141
 0.00628892    0.00855734    0.0430895        2.45124e-141  1.93227e-141
 ⋮                        

In [8]:
# empirical
Σ̂ = cov(X̃[:, 1:1000])
StatsBase.cov2cor!(Σ̂, diag(Σ̂))

1000×1000 Matrix{Float64}:
  1.0          0.703504      0.124672     …   0.0552838     0.03514
  0.703504     1.0           0.163654         0.0221817    -0.0150222
  0.124672     0.163654      1.0              0.000475529  -0.0737185
  0.0275016    0.0848931     0.569744        -0.0254694    -0.0678428
  0.0532734    0.104507      0.568687        -0.0205746    -0.0777318
  0.0752553    0.0385995     0.360167     …  -0.0366497    -0.0482012
  0.0787439    0.0297599     0.281991        -0.0259849    -0.0394486
  0.0864286    0.018689      0.211348        -0.00803859   -0.00935016
  0.0481485    0.0199564     0.166289         0.00240879    0.0172691
 -0.0121802   -0.0539207     0.156109        -0.0024882     0.0227963
 -0.0254511   -0.0622359     0.088712     …   0.0284792     0.0381364
 -0.0218163   -0.00796153    0.0956356        0.0065475     0.042407
  0.00364636   0.000783035   0.0726426       -0.0560607    -0.00648022
  ⋮                                       ⋱                
 -0.